In [6]:
import pandas as pd
import os
from datetime import datetime

pd.options.mode.chained_assignment = None  # 禁用SettingWithCopyWarning


####################    原始交易数据路径_格式化处理后    ####################
input_path = r"C:\Users\yhqhzz\Desktop\股指数据\交易信号标注\trading_data\账户二_0225-0317.xlsx"
####################    进行品种分类并创建多个文件储存的地址路径    ####################
output_path = r"C:\Users\yhqhzz\Desktop\股指数据\交易信号标注\trading_signal_data\账户二"



def generate_filename(contract_code, contract_num, start_date, end_date):
    # 文件名生成规则：
    # 1. 截取合约编码第9-10位并去除前导零
    # 2. 组合M标识、合约编码特征和日期范围
    part1 = contract_code[8:10].lstrip('0')  # 索引8-9对应第9-10个字符
    part2 = 'M'                             # 固定标识符
    part3 = contract_code[-6]               # 倒数第6位即正数第(len-5)位
    part4 = contract_code[-4:]              # 最后四位字符
    part5 = str(contract_num)[-4:].zfill(4) # 合约代码末四位补零
    date_str = f"{start_date}-{end_date}"    # 日期范围格式化
    return f"{part1}{part2}-{part3}-{part4}-{part5}-{date_str}"

def process_file(input_path, output_folder):
    # 读取并预处理原始数据
    df = pd.read_excel(input_path)
    
    # 列名重命名（基于列位置索引）
    df.rename(columns={
        df.columns[3]: '300ETF价格',  # 第4列 -> 300ETF价格
        df.columns[16]: 'IO价格',     # 第17列 -> IO价格
        df.columns[8]: '价格'         # 第9列 -> 价格
    }, inplace=True)
    
    # 按合约编码分组处理
    grouped = df.groupby('合约编码')
    
    for code, group in grouped:
        # 深拷贝当前分组数据避免警告
        group = group.copy()
        
        # 日期列转换与清洗
        group.loc[:, '报单日期'] = pd.to_datetime(
            group['报单日期'],
            format='%Y%m%d',    # 原始日期格式为YYYYMMDD
            errors='coerce'      # 无效日期转为NaT
        )
        
        # 有效日期范围计算
        valid_dates = group[group['报单日期'].notna()]
        start_date = valid_dates['报单日期'].min().strftime('%Y%m%d')
        end_date = valid_dates['报单日期'].max().strftime('%Y%m%d')
        
        # 文件名生成与文件保存
        sample_num = group['合约代码'].iloc[0]  # 取组内第一个合约代码
        filename = generate_filename(str(code), sample_num, start_date, end_date) + ".xlsx"
        
        # 按交易时间排序后保存
        sorted_group = group.sort_values(by='交易时间')
        os.makedirs(output_folder, exist_ok=True)
        output_path = os.path.join(output_folder, filename)
        sorted_group.to_excel(output_path, index=False)
        print(f"成功生成文件: {filename}")

if __name__ == "__main__":
    # 输入输出路径配置
    input_file = input_path  # 原始数据路径
    output_dir = output_path  # 输出目录
    import warnings
    warnings.filterwarnings("ignore")  # 屏蔽所有Python
    # 执行处理流程
    print(f"开始处理Excel文件{input_path}")
    process_file(input_file, output_dir)
    print("文件处理完成！结果保存在：", output_dir)


开始处理Excel文件C:\Users\yhqhzz\Desktop\股指数据\交易信号标注\trading_data\账户二_0225-0317.xlsx
成功生成文件: 3M-C-4000-3950-20250228-20250228.xlsx
成功生成文件: 3M-C-4100-4050-20250227-20250307.xlsx
成功生成文件: 3M-C-4200-4150-20250227-20250314.xlsx
成功生成文件: 3M-C-4300-4250-20250228-20250306.xlsx
成功生成文件: 3M-P-3900-3850-20250225-20250313.xlsx
成功生成文件: 3M-P-4000-3950-20250225-20250314.xlsx
成功生成文件: 3M-P-4100-4050-20250314-20250317.xlsx
成功生成文件: 4M-C-4100-4000-20250310-20250314.xlsx
成功生成文件: 4M-C-4200-4100-20250314-20250314.xlsx
成功生成文件: 4M-P-4000-3900-20250313-20250314.xlsx
文件处理完成！结果保存在： C:\Users\yhqhzz\Desktop\股指数据\交易信号标注\trading_signal_data\账户二


In [7]:
import os

# 获取文件夹中的所有文件名
files = os.listdir(output_dir)

# 遍历文件名并删除包含 "sort" 和 "_2" 的文件
for file in files:
    if "sort" in file or "_2" in file:
        file_path = os.path.join(output_dir, file)
        os.remove(file_path)
        print(f"已删除文件: {file}")

已删除文件: 3M-C-4000-3950_0225-0317_2.xlsx
已删除文件: 3M-C-4000-3950_0225-0317_2_sort.xlsx
已删除文件: 3M-C-4100-4050_0225-0317_2.xlsx
已删除文件: 3M-C-4100-4050_0225-0317_2_sort.xlsx
已删除文件: 3M-C-4200-4150_0225-0317_2.xlsx
已删除文件: 3M-C-4200-4150_0225-0317_2_sort.xlsx
已删除文件: 3M-C-4300-4250_0225-0317_2.xlsx
已删除文件: 3M-C-4300-4250_0225-0317_2_sort.xlsx
已删除文件: 3M-P-3900-3850_0225-0317_2.xlsx
已删除文件: 3M-P-3900-3850_0225-0317_2_sort.xlsx
已删除文件: 3M-P-4000-3950_0225-0317_2.xlsx
已删除文件: 3M-P-4000-3950_0225-0317_2_sort.xlsx
已删除文件: 3M-P-4100-4050_0225-0317_2.xlsx
已删除文件: 3M-P-4100-4050_0225-0317_2_sort.xlsx
已删除文件: 4M-C-4100-4000_0225-0317_2.xlsx
已删除文件: 4M-C-4100-4000_0225-0317_2_sort.xlsx
已删除文件: 4M-C-4200-4100_0225-0317_2.xlsx
已删除文件: 4M-C-4200-4100_0225-0317_2_sort.xlsx
已删除文件: 4M-P-4000-3900_0225-0317_2.xlsx
已删除文件: 4M-P-4000-3900_0225-0317_2_sort.xlsx
